In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (SparkSession.builder
         .master("spark://spark-master:7077")
         .appName("delta-test")
         .getOrCreate())

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /opt/spark/ivy/cache
The jars for the packages stored in: /opt/spark/ivy/jars
org.apache.spark#spark-hadoop-cloud_2.13 added as a dependency
io.delta#delta-spark_2.13 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-4.0_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-aae4024a-0675-4a47-9284-e762b705dfaa;1.0
	confs: [default]
	found org.apache.spark#spark-hadoop-cloud_2.13;4.0.1 in central
	found org.apache.hadoop#hadoop-client-runtime;3.4.1 in central
	found org.apache.hadoop#hadoop-client-api;3.4.1 in central
	found org.xerial.snappy#snappy-java;1.1.10.7 in central
	found org.slf4j#slf4j-api;2.0.16 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.hadoop#hadoop-aws;3.4.1 in central
	found org.wildfly.openssl#wildfly-openssl;2.2.5.Final in central
	found s

In [5]:
df = spark.read.option("header","true").csv("s3a://bronze/flights/")
df.show(5, truncate=False)

+--------+-----+--------+------+-----------+
|date    |delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|6    |602     |ABE   |ATL        |
|01020600|-8   |369     |ABE   |DTW        |
|01021245|-2   |602     |ABE   |ATL        |
|01020605|-4   |602     |ABE   |ATL        |
|01031245|-4   |602     |ABE   |ATL        |
+--------+-----+--------+------+-----------+
only showing top 5 rows


In [6]:
from pyspark.sql.functions import current_date
df_silver = df.withColumn("ingestion_date", current_date())

In [7]:
#Crear la database en el catálogo de Delta. En delta se llaman Databases y en iceberg Namespaces
spark.sql("CREATE DATABASE IF NOT EXISTS silver LOCATION 's3a://lakehouse/silver'")

26/01/11 16:50:38 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
26/01/11 16:50:38 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.18.0.6
26/01/11 16:50:38 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
26/01/11 16:50:39 WARN ObjectStore: Failed to get database silver, returning NoSuchObjectException
26/01/11 16:50:39 WARN ObjectStore: Failed to get database silver, returning NoSuchObjectException
26/01/11 16:50:39 WARN ObjectStore: Failed to get database silver, returning NoSuchObjectException


DataFrame[]

In [8]:
spark.sql("SHOW DATABASES").show(truncate=False)

+---------+
|namespace|
+---------+
|default  |
|silver   |
+---------+



In [9]:
spark.sql("SHOW TABLES IN silver").show(truncate=False)

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [10]:
df_silver.write.format("delta").mode("overwrite").saveAsTable("silver.flights")

26/01/11 16:51:18 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`silver`.`flights` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
26/01/11 16:51:18 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
26/01/11 16:51:18 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist


In [9]:
spark.sql("SHOW TABLES IN silver").show(truncate=False)

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|silver   |flights  |false      |
+---------+---------+-----------+



In [10]:
df_from_delta = spark.read \
    .format("delta") \
    .table("silver.flights")

In [11]:
df_from_delta.show()

26/01/11 13:37:46 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

+--------+-----+--------+------+-----------+--------------+
|    date|delay|distance|origin|destination|ingestion_date|
+--------+-----+--------+------+-----------+--------------+
|02151800|  108|     290|   ORD|        MSP|    2026-01-11|
|02151800|  142|     772|   ORD|        DEN|    2026-01-11|
|02151303|   16|    1516|   ORD|        LAX|    2026-01-11|
|02151157|    7|    1316|   ORD|        LAS|    2026-01-11|
|02151818|   55|    1511|   ORD|        PDX|    2026-01-11|
|02151033|   12|     873|   ORD|        MCO|    2026-01-11|
|02150941|    0|    1499|   ORD|        SNA|    2026-01-11|
|02151320|   17|    1604|   ORD|        SFO|    2026-01-11|
|02151804|    2|    1497|   ORD|        SAN|    2026-01-11|
|02152000|   17|     119|   ORD|        GRR|    2026-01-11|
|02151502|   15|     260|   ORD|        DSM|    2026-01-11|
|02151935|   73|     879|   ORD|        TPA|    2026-01-11|
|02151315|   -1|     879|   ORD|        TPA|    2026-01-11|
|02150957|    0|     939|   ORD|        

In [16]:
spark.stop()